<a href="https://colab.research.google.com/github/peterbmob/CH-PFC/blob/main/PH_PFC_paper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# untitled124_doping_stress.py
# Extends CH–PFC with dopant-aware free energy (regular solution + clustering)
# and computes composition-induced eigenstrain/stress from A(c).

import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from tqdm.auto import tqdm # Import tqdm for progress bar
from scipy.signal import find_peaks # Import find_peaks

plt.rcParams.update({
    "figure.figsize": (6, 4),
    "image.cmap": "viridis",
    "axes.spines.top": False,
    "axes.spines.right": False,
})
rng = np.random.default_rng(42)
SQRT3 = np.sqrt(3.0)

@dataclass
class Params:
    # --- PFC / CH (Balakrishna & Carter, Eq. (2)) ---
    r: float = -0.2
    gamma: float = 1.0
    psi_bar: float = 0.2
    xi: float = 1.0

    # composition-dependent transform (Eq. (4)): theta(c) = theta0 + dtheta * c
    theta0: float = np.pi/3   # hex reference
    dtheta: float = np.pi/6   # to square at c=1

    # numerics
    dx: float = 1.0
    dt_psi: float = 0.005
    dt_c: float = 0.001
    n_relax: int = 100
    kappa: float = 2.0

    # --- Thermodynamics (from LFP free-energy report) ---
    R_gas: float = 8.314
    T: float = 298.0
    omega: float = 1.2e4      # J/mol
    alpha1: float = 0 #1.1e3     # J/mol (native clustering)
    y1: float = 0.62
    w1: float = 0.05
    alpha2: float = 0 #4.0e3     # J/mol at unit dopant (dopant-enhanced clustering)
    y2: float = 0.60
    w2: float = 0.03
    C_dopant_scalar: float = 0 #0.10  # default uniform dopant
    fe_scale: float = None    # default: 1/(RT)

    # --- Elasticity for stress post-processing ---
    E_modulus: float = 100e9  # Pa
    nu: float = 0.3
    plane_stress: bool = True

P = Params()
if P.fe_scale is None:
    P.fe_scale = 1.0/(P.R_gas*P.T)

# --------- finite differences ----------
def laplacian(f, dx=P.dx):
    return (np.roll(f,-1,0)+np.roll(f,1,0)+np.roll(f,-1,1)+np.roll(f,1,1)-4.0*f)/(dx*dx)
def dxy(f, dx=P.dx):
    fpp = np.roll(np.roll(f,-1,0),-1,1)
    fpm = np.roll(np.roll(f,-1,0), 1,1)
    fmp = np.roll(np.roll(f, 1,0),-1,1)
    fmm = np.roll(np.roll(f, 1,0), 1,1)
    return (fpp - fpm - fmp + fmm)/(4.0*dx*dx)
def dxx(f, dx=P.dx):
    return (np.roll(f,-1,1)+np.roll(f,1,1)-2.0*f)/(dx*dx)
def dyy(f, dx=P.dx):
    return (np.roll(f,-1,0)+np.roll(f,1,0)-2.0*f)/(dx*dx)

# --------- A(c), ∇_c^2 and derivatives (Balakrishna & Carter Eqs. (3)-(4)) ----------
def A_components(c):
    theta = P.theta0 + P.dtheta * c
    A11 = np.ones_like(c)
    A12 = (2.0/SQRT3)*np.cos(theta) - 1.0/SQRT3
    A22 = (2.0/SQRT3)*np.sin(theta)
    return A11, A12, A22, theta

def dA_dc_components(c, theta=None):
    if theta is None:
        theta = P.theta0 + P.dtheta * c
    dtheta = P.dtheta
    dA11 = np.zeros_like(c)
    dA12 = (2.0/SQRT3)*(-np.sin(theta))*dtheta
    dA22 = (2.0/SQRT3)*( np.cos(theta))*dtheta
    return dA11, dA12, dA22

def Lc_apply(f, c):
    A11, A12, A22, _ = A_components(c)
    f_xx, f_yy, f_xy = dxx(f), dyy(f), dxy(f)
    return (P.xi**2)*((A11*A11 + A12*A12)*f_xx + (A22*A22)*f_yy + 2.0*A12*A22*f_xy)

def dLc_dc_apply(f, c):
    A11, A12, A22, theta = A_components(c)
    dA11, dA12, dA22 = dA_dc_components(c, theta)
    f_xx, f_yy, f_xy = dxx(f), dyy(f), dxy(f)
    return (P.xi**2)*(2.0*(A11*dA11 + A12*dA12)*f_xx + 2.0*(A22*dA22)*f_yy
                      + 2.0*(dA12*A22 + A12*dA22)*f_xy)

# --------- δF/δψ with fast relaxation (Eq. (6)) ----------
def deltaF_delta_psi(psi, c):
    Lpsi  = Lc_apply(psi, c)
    L2psi = Lc_apply(Lpsi, c)
    op_psi = P.r*psi + (psi + 2.0*Lpsi + L2psi)
    return P.gamma*op_psi + psi**3

def relax_psi(psi, c, n_steps=None, dt=None):
    if n_steps is None: n_steps = P.n_relax
    if dt is None: dt = P.dt_psi
    for _ in range(n_steps):
        g = deltaF_delta_psi(psi, c)
        g_mean = g.mean()
        psi = psi + dt * (-(g - g_mean))
        psi = psi - psi.mean() + P.psi_bar
    return psi

# --------- Free energy with dopant (from LFP report) ----------
EPS = 1e-9
def clamp01(x): return np.clip(x, EPS, 1.0-EPS)

def free_energy_density_dimless(c, C_dop=None):
    c = clamp01(c)
    if C_dop is None: C_dop = P.C_dopant_scalar
    RT = P.R_gas*P.T
    x = c
    ent = RT*(x*np.log(x) + (1-x)*np.log(1-x))
    nonideal = P.omega*x*(1-x)
    def cluster(alpha, y, w):
        g = np.exp(-((x-y)**2)/(2*w*w+1e-30))
        return alpha * x*(1-x) * g
    native = cluster(P.alpha1, P.y1, P.w1)
    dopant = cluster(P.alpha2, P.y2, P.w2)
    dG = ent + nonideal - native - C_dop*dopant
    return P.fe_scale * dG

def d_free_energy_dc_dimless(c, C_dop=None):
    c = clamp01(c)
    if C_dop is None: C_dop = P.C_dopant_scalar
    RT = P.R_gas*P.T
    x = c
    dent = RT*(np.log(x) - np.log(1-x))
    dnon = P.omega*(1-2*x)
    def d_cluster(alpha, y, w):
        g = np.exp(-((x-y)**2)/(2*w*w+1e-30))
        return alpha*g*((1-2*x) - (x*(1-x))*(x-y)/(w*w+1e-30))
    return P.fe_scale*(dent + dnon - d_cluster(P.alpha1,P.y1,P.w1)
                       - C_dop*d_cluster(P.alpha2,P.y2,P.w2))

# --------- μ(c) and CH update (Eq. (5)) ----------
def chemical_potential(c, psi, C_dop=None):
    mu = d_free_energy_dc_dimless(c, C_dop=C_dop) - P.kappa*laplacian(c)
    tmp = dLc_dc_apply(psi, c)
    coupling = 2.0*P.gamma*psi*(tmp + Lc_apply(tmp, c))
    return mu + coupling

def step_CH(c, psi, C_dop=None, dt=None):
    if dt is None: dt = P.dt_c
    mu = chemical_potential(c, psi, C_dop=C_dop)
    c = c + dt * laplacian(mu)
    return np.clip(c, 0.0, 1.0)

# --------- helpers ----------
def bandpass_noise(nx, ny, k0=1.0, rel_bw=0.15, amp=0.12):
    eta = rng.standard_normal((ny,nx))
    E = np.fft.rfftn(eta)
    ky = np.fft.fftfreq(ny, d=1.0) * 2*np.pi
    kx = np.fft.rfftfreq(nx, d=1.0) * 2*np.pi
    KX, KY = np.meshgrid(kx, ky, indexing='xy')
    K = np.sqrt(KX**2 + KY**2)
    W = np.exp(-((K-k0)**2)/(2*(rel_bw*k0+1e-12)**2))
    psi0 = amp*np.fft.irfftn(E*W, s=(ny,nx))
    psi0 = psi0 - psi0.mean() + P.psi_bar
    return psi0

def initialize_psi_lattice(shape, k0=1.0, pattern="hexagonal"):
    ny, nx = shape
    Y, X = np.mgrid[0:ny, 0:nx] * P.dx
    psi = np.zeros(shape)
    if pattern == "hexagonal":
        # Sum of three waves with 120 degree separation
        angle1 = 0
        angle2 = 2*np.pi/3
        angle3 = 4*np.pi/3
        psi += np.cos(k0 * (X * np.cos(angle1) + Y * np.sin(angle1)))
        psi += np.cos(k0 * (X * np.cos(angle2) + Y * np.sin(angle2)))
        psi += np.cos(k0 * (X * np.cos(angle3) + Y * np.sin(angle3)))
    elif pattern == "square":
        # Sum of two orthogonal waves
        psi += np.cos(k0 * X)
        psi += np.cos(k0 * Y)
    else:
        raise ValueError("Unknown pattern type")

    psi = psi - psi.mean() + P.psi_bar
    return psi

def initialize_psi_random(shape, value_range=(-0.1, 0.5)):
    """
    Initializes psi with random values within a specified range, centered around psi_bar.

    Parameters
    ----------
    shape : tuple
        Shape (ny, nx) of the grid.
    value_range : tuple
        (min_val, max_val) for the random values before centering around psi_bar.
    """
    ny, nx = shape
    min_val, max_val = value_range
    # Generate random numbers uniformly distributed between 0 and 1
    random_numbers = rng.uniform(0, 1, size=(ny, nx))

    # Scale and shift to the desired range [min_val, max_val]
    psi = min_val + random_numbers * (max_val - min_val)

    # Center the distribution around psi_bar
    current_mean = np.mean(psi)
    psi = psi - current_mean + P.psi_bar

    return psi


def show_field(field, title="", cmap="viridis"):
    plt.figure(); plt.imshow(field, origin='lower', cmap=cmap)
    plt.colorbar(shrink=0.8); plt.title(title); plt.tight_layout(); plt.show()

# --------- dopant maps ----------
def make_dopant_field(shape, kind="uniform", value=0.1):
    ny, nx = shape
    if kind=="uniform":
        return np.full((ny,nx), float(value))
    elif kind=="gradient-x":
        x = np.linspace(0,1,nx); return np.tile((value*x)[None,:], (ny,1))
    elif kind=="spot":
        Y,X = np.mgrid[0:ny,0:nx]; cx,cy,rad = nx//2, ny//2, min(nx,ny)/6
        R2 = (X-cx)**2 + (Y-cy)**2; return value*np.exp(-R2/(2*rad**2))
    else:
        raise ValueError("Unknown dopant field kind")

# --------- eigenstrain & stress from A(c) ----------
def eigenstrain_from_c(c):
    A11, A12, A22, _ = A_components(c)
    # J = [[1, A12], [0, A22]]; C = J^T J
    C11 = 1.0
    C12 = A12
    C22 = A12*A12 + A22*A22
    exx = 0.5*(C11 - 1.0) * np.ones_like(c) # Ensure exx has same shape as c
    eyy = 0.5*(C22 - 1.0)
    exy = 0.5*(C12) * np.ones_like(c) # Ensure exy has same shape as c
    return exx, eyy, exy

def stress_from_eigenstrain(exx, eyy, exy):
    E, nu = P.E_modulus, P.nu
    mu = E/(2*(1+nu))
    if P.plane_stress:
        D = (E/(1.0 - nu**2))*np.array([[1.0, nu, 0.0],
                                        [nu, 1.0, 0.0],
                                        [0.0, 0.0, (1.0 - nu)/2.0]])
    else:
        lam = E*nu/((1+nu)*(1-2*nu))
        D = np.array([[lam+2*mu, lam, 0.0],
                      [lam, lam+2*mu, 0.0],
                      [0.0, 0.0, mu]])
    gamma_xy = 2.0*exy
    e_stack = np.stack([exx, eyy, gamma_xy], axis=0)  # (3, ny, nx)
    s = D @ e_stack.reshape(3, -1)
    ny, nx = exx.shape
    sig_xx = s[0].reshape(ny,nx)
    sig_yy = s[1].reshape(ny,nx)
    sig_xy = s[2].reshape(ny,nx)
    return sig_xx, sig_yy, sig_xy
def stress_invariants(sig_xx, sig_yy, sig_xy):
    sigma_h = 0.5*(sig_xx + sig_yy)
    sigma_vm = np.sqrt(sig_xx**2 - sig_xx*sig_yy + sig_yy**2 + 3.0*sig_xy**2 + 1e-30)
    return sigma_h, sigma_vm

# --------- workflow ----------
def demo_build_initial_profile(nx=200, ny=30):
    c = np.zeros((ny,nx)); c[:, :40] = 1.0; c[:, 160:] = 1.0
    orig_gamma = P.gamma; P.gamma = 0.0
    for _ in range(400): c = step_CH(c, np.zeros_like(c), C_dop=None, dt=0.003)
    P.gamma = orig_gamma
    return c

def run_quasistatic_CHPFC(c, n_steps=200, snapshot_every=20, C_dop_field=None, psi_init_method="noise", psi_init_params={}):
    ny, nx = c.shape
    if psi_init_method == "noise":
        psi = bandpass_noise(nx, ny, **psi_init_params)
    elif psi_init_method == "lattice":
        psi = initialize_psi_lattice((ny,nx), **psi_init_params)
    elif psi_init_method == "random":
        psi = initialize_psi_random((ny,nx), **psi_init_params)
    else:
        raise ValueError("Unknown psi initialization method")

    snapshots = []
    for step in tqdm(range(n_steps), desc="Running simulation"): # Added tqdm progress bar
        psi = relax_psi(psi, c, n_steps=P.n_relax, dt=P.dt_psi)
        c = step_CH(c, psi, C_dop=C_dop_field, dt=P.dt_c)
        if step % snapshot_every == 0:
            snapshots.append((step, c.copy(), psi.copy()))
    return c, psi, snapshots

def compute_and_plot_stress(c, title_prefix=""):
    exx, eyy, exy = eigenstrain_from_c(c)
    sx, sy, sxy = stress_from_eigenstrain(exx, eyy, exy)
    sh, svm = stress_invariants(sx, sy, sxy)

    fig, axes = plt.subplots(2, 3, figsize=(12,6))
    im0=axes[0,0].imshow(exx,origin='lower'); axes[0,0].set_title(f"{title_prefix} ε_xx")
    im1=axes[0,1].imshow(eyy,origin='lower'); axes[0,1].set_title(f"{title_prefix} ε_yy")
    im2=axes[0,2].imshow(exy,origin='lower'); axes[0,2].set_title(f"{title_prefix} ε_xy")
    fig.colorbar(im0, ax=axes[0,0], shrink=0.7); fig.colorbar(im1, ax=axes[0,1], shrink=0.7); fig.colorbar(im2, ax=axes[0,2], shrink=0.7)

    im3=axes[1,0].imshow(sh/1e6,origin='lower'); axes[1,0].set_title(f"{title_prefix} σ_h (MPa)")
    im4=axes[1,1].imshow(svm/1e6,origin='lower'); axes[1,1].set_title(f"{title_prefix} σ_VM (MPa)")
    im5=axes[1,2].imshow(c,origin='lower', vmin=0, vmax=1); axes[1,2].set_title(f"{title_prefix} c (Li fraction)")
    fig.colorbar(im3, ax=axes[1,0], shrink=0.7); fig.colorbar(im4, ax=axes[1,1], shrink=0.7); fig.colorbar(im5, ax=axes[1,2], shrink=0.7)
    plt.tight_layout(); plt.show()


# ================= PLOTTING HELPERS =================
def plot_concentration_2d(c, title="Concentration c (2D)"):
    plt.figure(figsize=(6,3))
    im = plt.imshow(c, origin='lower', cmap='viridis', vmin=0, vmax=1)
    plt.colorbar(im, shrink=0.8, label='c')
    plt.title(title)
    plt.tight_layout(); plt.show()

def plot_concentration_profiles(c, dx=P.dx, title="Concentration profiles"):
    """
    Plots:
      - y-averaged profile <c>(x) across the cell
      - centerline profile c(x, y=ny//2)
    """
    ny, nx = c.shape
    x = np.arange(nx) * dx
    c_avg_x = c.mean(axis=0)
    c_mid   = c[ny//2, :]             # centerline along x (as in your original)

    plt.figure(figsize=(5,2))
    plt.plot(x, c_avg_x, label=r'$\langle c \rangle_y(x)$', lw=2)
    plt.plot(x, c_mid,   label=r'$c(x, y_{mid})$', lw=1.5, alpha=0.9)
    plt.xlabel("x")
    plt.ylabel("c (Li fraction)")
    plt.title(title)
    plt.grid(True, alpha=0.35)
    plt.legend()
    plt.tight_layout(); plt.show()

def plot_psi(psi, title="Peak-density field ψ (2D)"):
    plt.figure(figsize=(6,3))
    im = plt.imshow(psi, origin='lower', cmap='viridis')
    plt.colorbar(im, shrink=0.8, label='ψ')
    plt.title(title)
    plt.tight_layout(); plt.show()

def plot_reciprocal_lattice(psi, title="|FFT(ψ)|^2 (reciprocal space)"):
    Pk = np.fft.fftshift(np.abs(np.fft.fft2(psi))**2)
    plt.figure(figsize=(4.5,4))
    im = plt.imshow(np.log10(Pk + 1e-12), origin='lower', cmap='magma')
    plt.colorbar(im, shrink=0.8, label='log10 power')
    plt.title(title)
    plt.tight_layout(); plt.show()

def plot_free_energy_of_mixing(c, C_dop_field=None, n_bins=100, n_curve=400,
                               title="Free energy of mixing vs c",
                               show_hist=True):
    """
    Plots:
      - Domain-binned <f_hat>(c) computed from the current fields (c, C_dop_field)
      - Model curve f_hat(c) for a homogeneous dopant level equal to mean(C_dop)
      - Optional histogram of the current c-distribution (scaled)

    Left y-axis: dimensionless free energy density f_hat = ΔG_mix / (RT).
    Right y-axis: dimensional ΔG_mix (J/mol).

    Parameters
    ----------
    c : 2D array
        Current composition field in [0,1].
    C_dop_field : None, scalar, or 2D array
        Dopant concentration. If None -> uses P.C_dopant_scalar.
    n_bins : int
        Number of bins for the binned domain-averaged curve.
    n_curve : int
        Resolution for the homogeneous f(c) model curve.
    """

    # Local dopant (broadcasts if scalar)
    C_local = P.C_dopant_scalar if C_dop_field is None else C_dop_field

    # Compute local f_hat field with the CURRENT c and C_dop
    f_loc = free_energy_density_dimless(c, C_dop=C_local)

    # Bin the domain-averaged f_hat as a function of c
    bins = np.linspace(0.0, 1.0, n_bins+1)
    centers = 0.5*(bins[:-1] + bins[1:])
    digit = np.digitize(np.clip(c, 0.0, 1.0), bins) - 1
    f_binned = np.full_like(centers, np.nan, dtype=float)

    for i in range(len(centers)):
        mask = (digit == i)
        if np.any(mask):
            f_binned[i] = np.mean(f_loc[mask])

    # Homogeneous model curve at mean dopant
    Cmean = (float(C_local) if np.ndim(C_local) == 0 else float(np.nanmean(C_local)))
    x = np.linspace(1e-6, 1-1e-6, n_curve)
    f_curve = free_energy_density_dimless(x, C_dop=Cmean)

    # Figure
    fig, ax = plt.subplots(figsize=(7, 3.2))

    # Optional c-histogram (scaled to fit visually)
    if show_hist:
        hist_vals, hist_edges = np.histogram(c, bins=bins)
        hist_vals = hist_vals / (hist_vals.max() + 1e-12)  # scale 0..1
        ax.fill_between(centers, 0, hist_vals * (np.nanmax(f_binned)*0.25 if np.isfinite(np.nanmax(f_binned)) else 0),
                        color='gray', alpha=0.2, step='mid', label='c histogram (scaled)')

    # Domain-binned curve (current state)
    ax.plot(centers, f_binned, 'o-', ms=3, lw=1.5, label=r'$\langle \hat f \rangle_{\Omega}(c)$ (current)')

    # Homogeneous model curve at <C_dop>
    ax.plot(x, f_curve, '--', lw=1.8, label=fr'model $\hat f(c)$ at $\langle C_{{dop}}\rangle$={Cmean:.3f}')

    # Cosmetic: mark clustering centers if you like
    ax.axvline(P.y1, color='C3', ls=':', lw=1.2, alpha=0.8)
    ax.axvline(P.y2, color='C4', ls=':', lw=1.2, alpha=0.8)

    ax.set_xlabel('c (Li fraction)')
    ax.set_ylabel(r'$\hat f(c)$ (dimensionless, $\hat f=\Delta G_{mix}/RT$)')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=9)

    # Dimensional right axis in J/mol
    ax2 = ax.twinx()
    y1, y2 = ax.get_ylim()
    ax2.set_ylim(y1 / P.fe_scale, y2 / P.fe_scale)
    ax2.set_ylabel(r'$\Delta G_{mix}(c)$ (J/mol)')

    plt.tight_layout(); plt.show()
def plot_chemical_potential(c, psi, C_dop_field=None, n_bins=100, n_curve=400,
                            decompose=False,
                            title="Chemical potential vs c",
                            show_hist=True):
    """
    Plots:
      - Domain-binned <mu_full>(c) computed from the current fields (c, psi, C_dop_field)
      - Homogeneous thermodynamic mu_mix(c) = d f_hat / dc at mean dopant
      - Optional decomposition (thermo / gradient / coupling) as domain-binned curves
      - Optional histogram of the current c distribution (scaled)

    Left y-axis: dimensionless chemical potential (same scaling as free energy, i.e. /RT).
    Right y-axis: dimensional (J/mol).

    Parameters
    ----------
    c : 2D array in [0,1]
    psi : 2D array
    C_dop_field : None | scalar | 2D array
    decompose : bool
        If True, also plot binned contributions:
          mu_mix = d_free_energy_dc_dimless(c),
          mu_grad = -kappa ∇²c,
          mu_cpl = 2γ ψ ( tmp + Lc(tmp,c) ) with tmp = (∂Lc/∂c)ψ.
    """

    # Dopant field (broadcasts if scalar)
    C_local = P.C_dopant_scalar if C_dop_field is None else C_dop_field

    # --- Full μ field (this is what CH uses), and decomposition terms ---
    mu_full = chemical_potential(c, psi, C_dop=C_local)

    mu_mix  = d_free_energy_dc_dimless(c, C_dop=C_local)
    mu_grad = - P.kappa * laplacian(c)
    tmp     = dLc_dc_apply(psi, c)
    mu_cpl  = 2.0 * P.gamma * psi * (tmp + Lc_apply(tmp, c))

    # --- Bin by composition c to get <μ>(c) curves ---
    bins = np.linspace(0.0, 1.0, n_bins+1)
    centers = 0.5*(bins[:-1] + bins[1:])
    digit = np.digitize(np.clip(c, 0.0, 1.0), bins) - 1

    def bin_avg(field):
        out = np.full_like(centers, np.nan, dtype=float)
        for i in range(len(centers)):
            m = (digit == i)
            if np.any(m):
                out[i] = np.nanmean(field[m])
        return out

    mu_full_b = bin_avg(mu_full)
    mu_mix_b  = bin_avg(mu_mix)
    mu_grad_b = bin_avg(mu_grad)
    mu_cpl_b  = bin_avg(mu_cpl)

    # --- Homogeneous curve at mean dopant ---
    Cmean = (float(C_local) if np.ndim(C_local) == 0 else float(np.nanmean(C_local)))
    x = np.linspace(1e-6, 1-1e-6, n_curve)
    mu_curve_mix = d_free_energy_dc_dimless(x, C_dop=Cmean)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(7, 3.2))

    # Optional histogram of c
    if show_hist:
        hist_vals, hist_edges = np.histogram(c, bins=bins)
        hist_vals = hist_vals / (hist_vals.max() + 1e-12)  # scale 0..1
        # scale histogram to 25% of the μ_full_b dynamic range, if finite
        scale = 0.25*(np.nanmax(mu_full_b) - np.nanmin(mu_full_b)) if np.isfinite(np.nanmax(mu_full_b)) else 0.0
        ax.fill_between(centers,
                        np.nanmin(mu_full_b) if np.isfinite(np.nanmin(mu_full_b)) else 0.0,
                        (np.nanmin(mu_full_b) + hist_vals*scale) if scale>0 else 0.0,
                        color='gray', alpha=0.15, step='mid', label='c histogram (scaled)')

    # Full μ (domain-binned) – what CH uses
    ax.plot(centers, mu_full_b, 'o-', ms=3, lw=1.6, color='C0',
            label=r'$\langle \hat\mu \rangle_\Omega(c)$ (full)')

    # Homogeneous thermodynamic μ(c) at ⟨C_dop⟩
    ax.plot(x, mu_curve_mix, '--', color='k', lw=1.8,
            label=fr'$\hat\mu_\mathrm{{mix}}(c)$ at $\langle C_{{dop}}\rangle$={Cmean:.3f}')

    if decompose:
        ax.plot(centers, mu_mix_b,  '-',  lw=1.2, color='C3', alpha=0.9, label=r'$\langle \hat\mu_\mathrm{mix}\rangle$')
        ax.plot(centers, mu_grad_b, '-',  lw=1.2, color='C2', alpha=0.9, label=r'$\langle \hat\mu_\mathrm{grad}\rangle$')
        ax.plot(centers, mu_cpl_b,  '-',  lw=1.2, color='C4', alpha=0.9, label=r'$\langle \hat\mu_\mathrm{cpl}\rangle$')

    ax.set_xlabel('c (Li fraction)')
    ax.set_ylabel(r'$\hat\mu(c)$ (dimensionless)')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=9)

    # Right axis in J/mol (multiply by RT = 1/fe_scale)
    ax2 = ax.twinx()
    y1, y2 = ax.get_ylim()
    ax2.set_ylim(y1 / P.fe_scale, y2 / P.fe_scale)
    ax2.set_ylabel(r'$\mu(c)$ (J/mol)')

    plt.tight_layout(); plt.show()

from scipy.ndimage import maximum_filter
from scipy.signal import find_peaks

def quantify_peaks(psi, dx=P.dx, prominence=0.1, distance=5):
    """
    Quantifies the degree of squareness of the lattice in the psi field
    by analyzing peaks in the reciprocal lattice.

    Parameters
    ----------
    psi : 2D array
        The psi field.
    dx : float
        Spatial step size.
    prominence : float
        Required prominence of peaks for scipy.signal.find_peaks.
    distance : int
        Required horizontal distance between peaks for scipy.signal.find_peaks.

    Returns
    -------
    dict
        A dictionary containing quantitative measures of squareness.
    """
    ny, nx = psi.shape
    Pk = np.fft.fftshift(np.abs(np.fft.fft2(psi))**2)

    # Find peaks in the power spectrum
    # Flatten the array to use find_peaks
    Pk_flat = Pk.ravel()
    # Find peaks in the flattened array
    peaks_indices_flat, properties = find_peaks(Pk_flat, prominence=prominence, distance=distance)

    # Convert flat indices back to 2D coordinates
    peaks_coords_flat = np.unravel_index(peaks_indices_flat, Pk.shape)
    peaks_coords = np.vstack(peaks_coords_flat).T # (n_peaks, 2) array of (y, x) coordinates

    # Exclude the central peak (DC component)
    center_y, center_x = ny // 2, nx // 2
    # Find index of the peak closest to the center
    distances_to_center = np.sqrt((peaks_coords[:, 0] - center_y)**2 + (peaks_coords[:, 1] - center_x)**2)
    if len(distances_to_center) > 0:
        central_peak_index = np.argmin(distances_to_center)
        # Remove the central peak
        peaks_coords = np.delete(peaks_coords, central_peak_index, axis=0)
        peaks_indices_flat = np.delete(peaks_indices_flat, central_peak_index)
        peak_magnitudes = Pk_flat[peaks_indices_flat]
    else:
        # No peaks found other than possibly the center
        return {
            "num_peaks_found": 0,
            "square_peak_count": 0,
            "hexagonal_peak_count": 0,
            "squareness_ratio": 0.0,
            "peak_distance_variation": np.nan,
            "peak_angle_variation": np.nan
        }


    # Sort peaks by magnitude (brightness) in descending order
    sorted_indices = np.argsort(peak_magnitudes)[::-1]
    sorted_peaks_coords = peaks_coords[sorted_indices]
    sorted_peak_magnitudes = peak_magnitudes[sorted_indices]

    # Analyze the top few brightest peaks (e.g., top 4 or 6)
    num_peaks_to_analyze = min(6, len(sorted_peaks_coords)) # Look at up to 6 brightest peaks
    analyzed_peaks_coords = sorted_peaks_coords[:num_peaks_to_analyze]
    analyzed_peak_magnitudes = sorted_peak_magnitudes[:num_peaks_to_analyze]

    # Convert pixel coordinates to wavevectors (approximate)
    # Need to map pixel coordinates (y, x) relative to center to (ky, kx)
    ky_vals = np.fft.fftfreq(ny, d=dx) * 2*np.pi
    kx_vals = np.fft.fftfreq(nx, d=dx) * 2*np.pi # Note: rfftfreq for rfftn

    # Get the k-space coordinates of the analyzed peaks
    analyzed_peaks_k = []
    for p_coord in analyzed_peaks_coords:
        py, px = p_coord
        # Map pixel index relative to center to k-value
        # The kx_vals and ky_vals arrays are already sorted for fftfreq
        # Need to handle the shift correctly
        k_y = ky_vals[py if py < ny // 2 else py - ny]
        k_x = kx_vals[px if px < nx // 2 else px - nx] # This is simplified, need to be careful with fftshift and rfft
        # A more direct way might be to get the k-values corresponding to the pixel indices after fftshift
        k_y_shifted = np.fft.fftshift(ky_vals)[py]
        k_x_shifted = np.fft.fftshift(kx_vals)[px]

        analyzed_peaks_k.append((k_x_shifted, k_y_shifted)) # Store as (kx, ky)


    analyzed_peaks_k = np.array(analyzed_peaks_k)

    # --- Quantitative Measures ---

    # 1. Number of dominant peaks (excluding center)
    num_peaks_found = len(peaks_coords)

    # 2. Check for square pattern (4 peaks at approx (k0,0), (-k0,0), (0,k0), (0,-k0))
    #    and hexagonal pattern (6 peaks with 60 deg separation)
    square_peak_count = 0
    hexagonal_peak_count = 0
    k_magnitudes = np.sqrt(analyzed_peaks_k[:, 0]**2 + analyzed_peaks_k[:, 1]**2)
    k0_approx = np.mean(k_magnitudes) if len(k_magnitudes) > 0 else np.nan

    if len(analyzed_peaks_k) >= 4 and not np.isnan(k0_approx):
         # Check for approximate square arrangement
         # Look for peaks close to axes
         angles = np.arctan2(analyzed_peaks_k[:, 1], analyzed_peaks_k[:, 0]) # angles in radians
         angles_deg = np.degrees(angles) % 360

         # Check if angles are close to 0, 90, 180, 270 deg
         square_angles = np.array([0, 90, 180, 270])
         angle_tolerance = 15 # degrees

         is_square_like = False
         if len(analyzed_peaks_k) >= 4:
             # Check if at least 4 peaks have angles close to square angles
             angles_diff = np.abs(angles_deg[:, None] - square_angles[None, :])
             min_angle_diff = np.min(angles_diff, axis=1)
             peaks_close_to_square_angles = np.sum(min_angle_diff < angle_tolerance)
             if peaks_close_to_square_angles >= 4:
                 is_square_like = True
                 square_peak_count = peaks_close_to_square_angles # More accurate count

         # Check for approximate hexagonal arrangement (6 peaks with 60 deg separation)
         hexagonal_angles = np.array([0, 60, 120, 180, 240, 300])
         if len(analyzed_peaks_k) >= 6:
             angles_diff_hex = np.abs(angles_deg[:, None] - hexagonal_angles[None, :])
             min_angle_diff_hex = np.min(angles_diff_hex, axis=1)
             peaks_close_to_hex_angles = np.sum(min_angle_diff_hex < angle_tolerance)
             if peaks_close_to_hex_angles >= 6:
                 hexagonal_peak_count = peaks_close_to_hex_angles


    # 3. Ratio of square peak intensity to total dominant peak intensity
    #    This is a simple measure, could be refined.
    squareness_ratio = 0.0
    if square_peak_count > 0:
         # Sum magnitudes of peaks identified as square-like
         square_like_peak_magnitudes = analyzed_peak_magnitudes[min_angle_diff < angle_tolerance]
         total_analyzed_magnitude = np.sum(analyzed_peak_magnitudes)
         if total_analyzed_magnitude > 1e-12:
             squareness_ratio = np.sum(square_like_peak_magnitudes) / total_analyzed_magnitude
         else:
              squareness_ratio = 0.0


    # 4. Variation in distances of the main peaks from the origin (should be ~k0)
    peak_distance_variation = np.std(k_magnitudes) / k0_approx if k0_approx > 1e-12 and len(k_magnitudes) > 1 else np.nan

    # 5. Variation in angles between main peaks (should be ~90 deg for square, ~60 for hex)
    #    This is harder to quantify robustly without pairing peaks.
    #    A simpler approach is to look at the deviation of angles from ideal square angles.
    if is_square_like and len(analyzed_peaks_k) >= 4:
         # Calculate angles between consecutive peaks if sorted by angle, or more robustly check angles between pairs
         # For simplicity, let's just report the min angle diff to ideal square angles for square-like peaks
         peak_angle_variation = np.mean(min_angle_diff[min_angle_diff < angle_tolerance]) if np.sum(min_angle_diff < angle_tolerance) > 0 else np.nan
    else:
         peak_angle_variation = np.nan


    results = {
        "num_peaks_found": num_peaks_found, # Total non-central peaks found
        "square_peak_count": square_peak_count, # Peaks close to square angles
        "hexagonal_peak_count": hexagonal_peak_count, # Peaks close to hexagonal angles
        "squareness_ratio": squareness_ratio, # Ratio of magnitude of square-like peaks to total analyzed peaks
        "peak_distance_variation": peak_distance_variation, # Std deviation of k-magnitudes / mean k-magnitude
        "peak_angle_variation": peak_angle_variation # Mean deviation from ideal square angles for square-like peaks
    }

    return results

# --- Example Usage ---
# Assuming psi_u is available from a simulation run
# squareness_metrics = quantify_squareness(psi_u)
# print("\nSquareness Metrics:")
# print(squareness_metrics)

# Example with gradient psi (if available)
# squareness_metrics_g = quantify_squareness(psi_g)
# print("\nSquareness Metrics (Gradient):")
# print(squareness_metrics_g)

# Tests for c=1 and c=0

In [ ]:
ny, nx = 363, 363
#c0 = demo_build_initial_profile(nx=nx, ny=ny)
c0 = np.full((ny,nx), 1.0) # Initialize with uniform concentration
C_uniform  = make_dopant_field(c0.shape, kind="uniform",    value=P.C_dopant_scalar)
#C_gradient = make_dopant_field(c0.shape, kind="gradient-x", value=0.2)
c_u, psi_u, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=100, snapshot_every=50, C_dop_field=C_uniform,  psi_init_method="lattice", psi_init_params={"pattern": "square"})
#c_g, psi_g, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=50, snapshot_every=50, C_dop_field=C_gradient,  psi_init_method="lattice", psi_init_params={"pattern": "hexagonal"})
#compute_and_plot_stress(c_u, "Uniform dopant")
#compute_and_plot_stress(c_g, "Gradient dopant")

In [ ]:
# After uniform dopant run:
plot_concentration_2d(c_u, title="c after CH–PFC (uniform dopant)")
plot_concentration_profiles(c_u, title="c profiles (uniform dopant)")
plot_psi(psi_u, title="ψ after CH–PFC (uniform dopant)")

plot_reciprocal_lattice(psi_u)   # optional lattice check
metrics = quantify_peaks(psi_u)
print("\npeak Metrics:")
print(metrics)
# After gradient dopant run:
#plot_concentration_2d(c_g, title="c after CH–PFC (dopant gradient)")
#plot_concentration_profiles(c_g, title="c profiles (dopant gradient)")
#plot_psi(psi_g, title="ψ after CH–PFC (dopant gradient)")
# plot_reciprocal_lattice(psi_g)   # optional lattice check

# Calculate local mixing free energy density
local_mixing_fe_density = free_energy_density_dimless(c_u, C_dop=C_uniform)

# Calculate average mixing free energy density over the domain
average_mixing_fe_density = np.mean(local_mixing_fe_density)

# Calculate total mixing free energy (sum over grid and multiply by cell area)
total_mixing_fe = np.sum(local_mixing_fe_density) * (P.dx * P.dx)

print(f"Average dimensionless mixing free energy density: {average_mixing_fe_density:.6f}")
print(f"Total dimensionless mixing free energy: {total_mixing_fe:.6f}")

# You can also convert this to J/mol if you multiply by RT
average_mixing_fe_dimensional = average_mixing_fe_density / P.fe_scale
total_mixing_fe_dimensional = total_mixing_fe / P.fe_scale

print(f"Average mixing free energy density (J/mol): {average_mixing_fe_dimensional:.2f}")
print(f"Total mixing free energy (J): {total_mixing_fe_dimensional:.2f}")

# Display the spatial distribution of the local mixing free energy density
show_field(local_mixing_fe_density, title="Local mixing free energy density (dimensionless)")

In [ ]:
plot_free_energy_of_mixing(c_u, C_dop_field=C_uniform,
                           title="Free energy of mixing vs c (uniform dopant)")

#plot_free_energy_of_mixing(c_g, C_dop_field=C_gradient,
#                           title="Free energy of mixing vs c (dopant gradient)")

In [ ]:
plot_chemical_potential(c_u, psi_u, C_dop_field=C_uniform,
                        decompose=True,
                        title="Chemical potential vs c (uniform dopant)")

#plot_chemical_potential(c_g, psi_g, C_dop_field=C_gradient,
#                        decompose=True,
#                        title="Chemical potential vs c (dopant gradient)")

In [ ]:
ny, nx = 145, 145
#c0 = demo_build_initial_profile(nx=nx, ny=ny)
c0 = np.full((ny,nx), 0.0) # Initialize with uniform concentration
C_uniform  = make_dopant_field(c0.shape, kind="uniform",    value=P.C_dopant_scalar)
#C_gradient = make_dopant_field(c0.shape, kind="gradient-x", value=0.2)
c_u, psi_u, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=100, snapshot_every=50, C_dop_field=C_uniform,  psi_init_method="lattice", psi_init_params={"pattern": "hexagonal"})
#c_g, psi_g, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=50, snapshot_every=200, C_dop_field=C_gradient,  psi_init_method="lattice", psi_init_params={"pattern": "hexagonal"})
#compute_and_plot_stress(c_u, "Uniform dopant")
#compute_and_plot_stress(c_g, "Gradient dopant")

In [ ]:
# After uniform dopant run:
plot_concentration_2d(c_u, title="c after CH–PFC (uniform dopant)")
plot_concentration_profiles(c_u, title="c profiles (uniform dopant)")
plot_psi(psi_u, title="ψ after CH–PFC (uniform dopant)")

plot_reciprocal_lattice(psi_u)   # optional lattice check
metrics = quantify_peaks(psi_u)
print("\npeak Metrics:")
print(metrics)

# After gradient dopant run:
#plot_concentration_2d(c_g, title="c after CH–PFC (dopant gradient)")
#plot_concentration_profiles(c_g, title="c profiles (dopant gradient)")
#plot_psi(psi_g, title="ψ after CH–PFC (dopant gradient)")
# plot_reciprocal_lattice(psi_g)   # optional lattice check

# Calculate local mixing free energy density
local_mixing_fe_density = free_energy_density_dimless(c_u, C_dop=C_uniform)

# Calculate average mixing free energy density over the domain
average_mixing_fe_density = np.mean(local_mixing_fe_density)

# Calculate total mixing free energy (sum over grid and multiply by cell area)
total_mixing_fe = np.sum(local_mixing_fe_density) * (P.dx * P.dx)

print(f"Average dimensionless mixing free energy density: {average_mixing_fe_density:.6f}")
print(f"Total dimensionless mixing free energy: {total_mixing_fe:.6f}")

# You can also convert this to J/mol if you multiply by RT
average_mixing_fe_dimensional = average_mixing_fe_density / P.fe_scale
total_mixing_fe_dimensional = total_mixing_fe / P.fe_scale

print(f"Average mixing free energy density (J/mol): {average_mixing_fe_dimensional:.2f}")
print(f"Total mixing free energy (J): {total_mixing_fe_dimensional:.2f}")

# Display the spatial distribution of the local mixing free energy density
show_field(local_mixing_fe_density, title="Local mixing free energy density (dimensionless)")

In [ ]:
plot_free_energy_of_mixing(c_u, C_dop_field=C_uniform,
                           title="Free energy of mixing vs c (uniform dopant)")

#plot_free_energy_of_mixing(c_g, C_dop_field=C_gradient,
#                           title="Free energy of mixing vs c (dopant gradient)")

In [ ]:
plot_chemical_potential(c_u, psi_u, C_dop_field=C_uniform,
                        decompose=True,
                        title="Chemical potential vs c (uniform dopant)")

#plot_chemical_potential(c_g, psi_g, C_dop_field=C_gradient,
#                        decompose=True,
#                        title="Chemical potential vs c (dopant gradient)")

# Diffuse interphase

In [ ]:
ny, nx = 30, 200
P.kappa=20
c0 = demo_build_initial_profile(nx=nx, ny=ny)
#c0 = np.full((ny,nx), 0.0) # Initialize with uniform concentration
C_uniform  = make_dopant_field(c0.shape, kind="uniform",    value=P.C_dopant_scalar)
#C_gradient = make_dopant_field(c0.shape, kind="gradient-x", value=0.2)
#c_u, psi_u, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=50, snapshot_every=50, C_dop_field=C_uniform,  psi_init_method="lattice", psi_init_params={"pattern": "square"})
c_u, psi_u, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=500, snapshot_every=50, C_dop_field=C_uniform,  psi_init_method="random")
#c_g, psi_g, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=50, snapshot_every=50, C_dop_field=C_gradient

#c_g, psi_g, _ = run_quasistatic_CHPFC(c0.copy(), n_steps=50, snapshot_every=50, C_dop_field=C_gradient,  psi_init_method="lattice", psi_init_params={"pattern": "hexagonal"})
#compute_and_plot_stress(c_u, "Uniform dopant")
#compute_and_plot_stress(c_g, "Gradient dopant")


Now I want to evolve the system in time allowing for simulations of phase simulation given a starting lattice with a certain Li concentration. I want to simulate the phase separation in time. How would I o that?

In [ ]:
# After uniform dopant run:
plot_concentration_2d(c_u, title="c after CH–PFC (uniform dopant)")
plot_concentration_profiles(c_u, title="c profiles (uniform dopant)")
plot_psi(psi_u, title="ψ after CH–PFC (uniform dopant)")
plot_reciprocal_lattice(psi_u)   # optional lattice check



# Calculate local mixing free energy density
local_mixing_fe_density = free_energy_density_dimless(c_u, C_dop=C_uniform)

# Calculate average mixing free energy density over the domain
average_mixing_fe_density = np.mean(local_mixing_fe_density)

# Calculate total mixing free energy (sum over grid and multiply by cell area)
total_mixing_fe = np.sum(local_mixing_fe_density) * (P.dx * P.dx)

print(f"Average dimensionless mixing free energy density: {average_mixing_fe_density:.6f}")
print(f"Total dimensionless mixing free energy: {total_mixing_fe:.6f}")

# You can also convert this to J/mol if you multiply by RT
average_mixing_fe_dimensional = average_mixing_fe_density / P.fe_scale
total_mixing_fe_dimensional = total_mixing_fe / P.fe_scale

print(f"Average mixing free energy density (J/mol): {average_mixing_fe_dimensional:.2f}")
print(f"Total mixing free energy (J): {total_mixing_fe_dimensional:.2f}")

# Display the spatial distribution of the local mixing free energy density
show_field(local_mixing_fe_density, title="Local mixing free energy density (dimensionless)")

In [ ]:
plot_free_energy_of_mixing(c_u, C_dop_field=C_uniform,
                           title="Free energy of mixing vs c (uniform dopant)")

#plot_free_energy_of_mixing(c_g, C_dop_field=C_gradient,
#                           title="Free energy of mixing vs c (dopant gradient)")

In [ ]:
plot_chemical_potential(c_u, psi_u, C_dop_field=C_uniform,
                        decompose=True,
                        title="Chemical potential vs c (uniform dopant)")

#plot_chemical_potential(c_g, psi_g, C_dop_field=C_gradient,
#                        decompose=True,
#                        title="Chemical potential vs c (dopant gradient)")

# More detailed analysis

In [ ]:
def plot_concentration_and_psi_contours(c, psi, title="Concentration with Psi Contours"):
    """
    Plots the concentration field as an image with psi contours overlaid.

    Parameters
    ----------
    c : 2D array
        The concentration field.
    psi : 2D array
        The psi field.
    title : str
        Title for the plot.
    """
    plt.figure(figsize=(6, 4))

    # Plot the concentration field as an image
    im = plt.imshow(c, origin='lower', cmap='viridis', vmin=0, vmax=1)
    plt.colorbar(im, shrink=0.8, label='c')

    # Plot contours of the psi field
    # You might need to adjust the contour levels based on the typical range of your psi values
    # Here we plot contours at certain positive psi values to highlight lattice sites
    contour_levels = np.linspace(psi.min(), psi.max(), 10) # Example: 10 contour levels across the range
    # Or specify specific levels, e.g., contour_levels = [0.2, 0.3, 0.4]
    plt.contour(psi, levels=contour_levels, colors='w', linewidths=0.8, alpha=0.7)


    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    plt.show()

# --- Example Usage ---
# Assuming c_u and psi_u are available from a simulation run
plot_concentration_and_psi_contours(c_u, psi_u, title="c and psi contours (Uniform dopant)")

In [ ]:
def plot_local_reciprocal_lattice(psi, x_center, window_width, title=""):
    """
    Plots the reciprocal lattice for a vertical strip of the psi field.

    Parameters
    ----------
    psi : 2D array
        The psi field.
    x_center : int
        The x-coordinate of the center of the vertical strip.
    window_width : int
        The width of the vertical strip.
    title : str
        Title for the plot.
    """
    ny, nx = psi.shape
    x_start = max(0, x_center - window_width // 2)
    x_end = min(nx, x_center + window_width // 2)

    if x_end - x_start < window_width:
        print(f"Warning: Window clipped at boundary. Actual width: {x_end - x_start}")

    local_psi = psi[:, x_start:x_end]

    # Apply a window function to reduce spectral leakage (e.g., a 2D Tukey window)
    # A simple 1D Tukey window applied to each column is a start.
    # More sophisticated 2D windowing might be needed for quantitative analysis.
    tukey_window = np.outer(np.ones(ny), np.hanning(x_end - x_start))
    local_psi_windowed = local_psi * tukey_window


    Pk = np.fft.fftshift(np.abs(np.fft.fft2(local_psi_windowed))**2)

    plt.figure(figsize=(4.5,4))
    im = plt.imshow(np.log10(Pk + 1e-12), origin='lower', cmap='magma')
    plt.colorbar(im, shrink=0.8, label='log10 power')
    # Adjust extent to reflect k-space
    ky = np.fft.fftfreq(ny, d=P.dx) * 2*np.pi
    kx = np.fft.fftfreq(x_end - x_start, d=P.dx) * 2*np.pi
    plt.xticks(np.arange(len(kx)), [f'{k:.1f}' for k in np.fft.fftshift(kx)])
    plt.yticks(np.arange(len(ky)), [f'{k:.1f}' for k in np.fft.fftshift(ky)])
    plt.xlabel('$k_x$')
    plt.ylabel('$k_y$')


    plt.title(title)
    plt.tight_layout(); plt.show()

# --- Analyze local symmetry across the interface ---
# Assuming c_u and psi_u from a simulation with an interface
# If you used demo_build_initial_profile, the interface is around x=40

# Define window parameters
window_width = 32 # pixels
x_positions = [20, 40, 60, 80, 100, 120, 140, 160, 180, 200] # x-coordinates for the center of each window

# Run local analysis for each position
for x_center in x_positions:
    plot_local_reciprocal_lattice(psi_u, x_center, window_width,
                                  title=f"Local Reciprocal Lattice at x={x_center} (Uniform dopant)")

# If you also ran a simulation with a gradient (e.g., c_g, psi_g)
# for x_center in x_positions:
#     plot_local_reciprocal_lattice(psi_g, x_center, window_width,
#                                   title=f"Local Reciprocal Lattice at x={x_center} (Gradient dopant)")

# Testing effect of parameters

In [ ]:
def tune_square_stability(r_val, gamma_val, n_steps=200):
    """
    Runs a short simulation at fixed c=1.0 to evaluate square lattice stability.

    Temporarily modifies P.r and P.gamma.

    Parameters
    ----------
    r_val : float
        The value of P.r to test.
    gamma_val : float
        The value of P.gamma to test.
    n_steps : int
        Number of simulation steps.
    """
    # Store original parameters
    original_r = P.r
    original_gamma = P.gamma

    # Temporarily set parameters
    P.r = r_val
    P.gamma = gamma_val

    ny, nx = 100, 100
    c_fixed_one = np.full((ny, nx), 1.0) # Fixed concentration at 1.0

    print(f"Running tuning simulation for r={P.r}, gamma={P.gamma}")

    try:
        # Run simulation at fixed c=1, initialize with square lattice
        # We might need to adjust n_relax or dt_psi for stability with new parameters,
        # but let's start with the current ones and see if it's stable.
        c_result, psi_result, _ = run_quasistatic_CHPFC(
            c_fixed_one.copy(),
            n_steps=n_steps,
            snapshot_every=n_steps + 1, # Only get final snapshot
            C_dop_field=None, # Assuming no dopant effect on this tuning
            psi_init_method="lattice",
            psi_init_params={"pattern": "square", "k0": 1.0/P.xi} # Use k0 related to P.xi
        )

        print(f"\n--- Results for r={P.r}, gamma={P.gamma} ---")
        plot_psi(psi_result, title=f"ψ at c=1 (r={P.r}, γ={P.gamma})")
        plot_reciprocal_lattice(psi_result, title=f"|FFT(ψ)|^2 at c=1 (r={P.r}, γ={P.gamma})")

    except Exception as e:
        print(f"\nSimulation failed for r={P.r}, gamma={P.gamma} with error: {e}")
    finally:
        # Restore original parameters
        P.r = original_r
        P.gamma = original_gamma
        print("Restored original parameters.")

# --- Example Tuning Calls ---
# You can uncomment and run these calls one by one, or in groups,
# to test different parameter combinations.

# tune_square_stability(-0.2, 1.0) # Original parameters

# Try more negative r
# tune_square_stability(-0.3, 1.0)
# tune_square_stability(-0.4, 1.0)

# Try larger gamma
# tune_square_stability(-0.2, 1.2)
# tune_square_stability(-0.2, 1.5)

# Try combinations
# tune_square_stability(-0.3, 1.2)
# tune_square_stability(-0.4, 1.5)

# Example with increased steps if pattern is slow to form
# tune_square_stability(-0.3, 1.2, n_steps=500)

In [ ]:
tune_square_stability(-0.2, 1.0) # Original parameters

In [ ]:
def compare_phase_stability_at_c0(n_steps=500, return_energies=False):
    """
    Runs simulations at fixed c=0.0 with hexagonal and square psi initializations
    and compares the resulting average mixing free energies.

    Parameters
    ----------
    n_steps : int
        Number of simulation steps for each run.
    return_energies : bool
        If True, returns (avg_fe_hex, avg_fe_square) instead of plotting/printing.
    """
    ny, nx = 100, 100
    c_fixed_zero = np.full((ny, nx), 0.0) # Fixed concentration at 0.0
    C_uniform_c0 = make_dopant_field(c_fixed_zero.shape, kind="uniform", value=P.C_dopant_scalar) # Use a uniform dopant field

    # --- Run with Hexagonal Initialization ---
    if not return_energies: print("\nRunning simulation with hexagonal initialization...")
    c_hex, psi_hex, _ = run_quasistatic_CHPFC(
        c_fixed_zero.copy(),
        n_steps=n_steps,
        snapshot_every=n_steps + 1, # Only get final snapshot
        C_dop_field=C_uniform_c0,
        psi_init_method="lattice",
        psi_init_params={"pattern": "hexagonal", "k0": 1.0/P.xi} # Use k0 related to P.xi
    )
    avg_fe_hex = np.mean(free_energy_density_dimless(c_hex, C_dop=C_uniform_c0))
    if not return_energies:
        print(f"Average dimensionless mixing free energy (Hexagonal init): {avg_fe_hex:.6f}")
        plot_psi(psi_hex, title=f"ψ at c=0 (Hexagonal init, r={P.r}, γ={P.gamma})")
        plot_reciprocal_lattice(psi_hex, title=f"|FFT(ψ)|^2 at c=0 (Hexagonal init, r={P.r}, γ={P.gamma})")


    # --- Run with Square Initialization ---
    if not return_energies: print("\nRunning simulation with square initialization...")
    c_square, psi_square, _ = run_quasistatic_CHPFC(
        c_fixed_zero.copy(),
        n_steps=n_steps,
        snapshot_every=n_steps + 1, # Only get final snapshot
        C_dop_field=C_uniform_c0,
        psi_init_method="lattice",
        psi_init_params={"pattern": "square", "k0": 1.0/P.xi} # Use k0 related to P.xi
    )
    avg_fe_square = np.mean(free_energy_density_dimless(c_square, C_dop=C_uniform_c0))
    if not return_energies:
        print(f"Average dimensionless mixing free energy (Square init): {avg_fe_square:.6f}")
        plot_psi(psi_square, title=f"ψ at c=0 (Square init, r={P.r}, γ={P.gamma})")
        plot_reciprocal_lattice(psi_square, title=f"|FFT(ψ)|^2 at c=0 (Square init, r={P.r}, γ={P.gamma})")

    if return_energies:
        return avg_fe_hex, avg_fe_square
    else:
        print("\nComparison Summary:")
        print(f"Average dimensionless mixing free energy (Hexagonal init): {avg_fe_hex:.6f}")
        print(f"Average dimensionless mixing free energy (Square init): {avg_fe_square:.6f}")
        if avg_fe_square < avg_fe_hex:
            print("-> Square initial condition led to a state with lower average mixing free energy at c=0.")
        elif avg_fe_hex < avg_fe_square:
            print("-> Hexagonal initial condition led to a state with lower average mixing free energy at c=0.")
        else:
            print("-> Average mixing free energies are approximately equal.")

# --- Example Call ---
# You can call this function after setting your desired P.r, P.gamma, P.xi in Params
# compare_phase_stability_at_c0(n_steps=500)

In [ ]:
compare_phase_stability_at_c0(n_steps=50)

In [ ]:
import pandas as pd

def screen_pfc_parameters_at_c0(r_val, gamma_val, omega_values, n_steps=500):
    """
    Screens combinations of P.omega values (keeping P.r and P.gamma fixed)
    to evaluate phase stability at c=0.0.

    Runs simulations at fixed c=0.0 with hexagonal and square psi initializations
    and compares the resulting average mixing free energies for each parameter set.

    Parameters
    ----------
    r_val : float
        Fixed P.r value to use.
    gamma_val : float
        Fixed P.gamma value to use.
    omega_values : list or array
        List of P.omega values to test.
    n_steps : int
        Number of simulation steps for each run in the comparison function.

    Returns
    -------
    pandas.DataFrame
        DataFrame containing the tested parameters and the resulting
        average mixing free energies for hexagonal and square initializations.
    """
    results = []
    original_omega = P.omega
    original_r = P.r
    original_gamma = P.gamma

    print(f"Starting parameter screening for Omega at c=0.0 (P.r={r_val}, P.gamma={gamma_val})...")

    # Temporarily set fixed P.r and P.gamma for this screening
    P.r = r_val
    P.gamma = gamma_val

    for omega_val in omega_values:
        # Temporarily set P.omega
        P.omega = omega_val

        try:
            # Run comparison and get energies
            avg_fe_hex, avg_fe_square = compare_phase_stability_at_c0(
                n_steps=n_steps,
                return_energies=True
            )
            results.append({
                "P.r": r_val,
                "P.gamma": gamma_val,
                "P.omega": omega_val,
                "Avg_FE_Hex": avg_fe_hex,
                "Avg_FE_Square": avg_fe_square,
                "FE_Diff_Hex_Minus_Square": avg_fe_hex - avg_fe_square # Positive means Square is lower energy (more stable mixing-wise)
            })
            print(f"  Tested omega={omega_val:.1f}. Avg FE Hex: {avg_fe_hex:.6f}, Avg FE Square: {avg_fe_square:.6f}")

        except Exception as e:
            print(f"  Simulation failed for omega={omega_val:.1f} with error: {e}")
            results.append({
                "P.r": r_val,
                "P.gamma": gamma_val,
                "P.omega": omega_val,
                "Avg_FE_Hex": np.nan,
                "Avg_FE_Square": np.nan,
                "FE_Diff_Hex_Minus_Square": np.nan
            })
        finally:
            # Restore original P.omega after each test
            P.omega = original_omega

    print("Parameter screening finished.")

    # Restore original P.r and P.gamma
    P.r = original_r
    P.gamma = original_gamma
    P.omega = original_omega # Ensure omega is also restored

    return pd.DataFrame(results)

# --- Example Screening Run for Omega at c=0 ---
# Define the ranges of parameters to test
r_fixed = P.r # Use current P.r
gamma_fixed = P.gamma # Use current P.gamma
omega_test_values_c0 = np.linspace(1.5e4, 2.0e4, 5) # Example: 5 values around current omega

# Run the screening
screening_results_omega_c0 = screen_pfc_parameters_at_c0(r_fixed, gamma_fixed, omega_test_values_c0, n_steps=50)

# Display the results, sorted by energy difference
# For c=0, we want Hexagonal to be more stable, meaning Avg_FE_Hex < Avg_FE_Square,
# which corresponds to FE_Diff_Hex_Minus_Square < 0.
display(screening_results_omega_c0.sort_values(by="FE_Diff_Hex_Minus_Square", ascending=True))

# Interpretation: A more NEGATIVE value of "FE_Diff_Hex_Minus_Square" means
# the Average Mixing Free Energy for the Hexagonal initial condition was LOWER,
# suggesting the parameters favor the hexagonal lattice more (in terms of mixing energy) at c=0.

In [ ]:
def compare_phase_stability_at_c1(n_steps=500, return_energies=False):
    """
    Runs simulations at fixed c=1.0 with hexagonal and square psi initializations
    and compares the resulting average mixing free energies.

    Parameters
    ----------
    n_steps : int
        Number of simulation steps for each run.
    return_energies : bool
        If True, returns (avg_fe_hex, avg_fe_square) instead of plotting/printing.
    """
    ny, nx = 100, 100
    c_fixed_one = np.full((ny, nx), 1.0) # Fixed concentration at 1.0
    C_uniform_c1 = make_dopant_field(c_fixed_one.shape, kind="uniform", value=P.C_dopant_scalar) # Use a uniform dopant field

    print(f"Comparing phase stability at c=1.0 for P.r={P.r}, P.gamma={P.gamma}, P.xi={P.xi}")

    # --- Run with Hexagonal Initialization ---
    if not return_energies: print("\nRunning simulation with hexagonal initialization...")
    c_hex, psi_hex, _ = run_quasistatic_CHPFC(
        c_fixed_one.copy(),
        n_steps=n_steps,
        snapshot_every=n_steps + 1, # Only get final snapshot
        C_dop_field=C_uniform_c1,
        psi_init_method="lattice",
        psi_init_params={"pattern": "hexagonal", "k0": 1.0/P.xi} # Use k0 related to P.xi
    )
    avg_fe_hex = np.mean(free_energy_density_dimless(c_hex, C_dop=C_uniform_c1))
    if not return_energies:
        print(f"Average dimensionless mixing free energy (Hexagonal init): {avg_fe_hex:.6f}")
        plot_psi(psi_hex, title=f"ψ at c=1 (Hexagonal init, r={P.r}, γ={P.gamma})")
        plot_reciprocal_lattice(psi_hex, title=f"|FFT(ψ)|^2 at c=1 (Hexagonal init, r={P.r}, γ={P.gamma})")


    # --- Run with Square Initialization ---
    if not return_energies: print("\nRunning simulation with square initialization...")
    c_square, psi_square, _ = run_quasistatic_CHPFC(
        c_fixed_one.copy(),
        n_steps=n_steps,
        snapshot_every=n_steps + 1, # Only get final snapshot
        C_dop_field=C_uniform_c1,
        psi_init_method="lattice",
        psi_init_params={"pattern": "square", "k0": 1.0/P.xi} # Use k0 related to P.xi
    )
    avg_fe_square = np.mean(free_energy_density_dimless(c_square, C_dop=C_uniform_c1))
    if not return_energies:
        print(f"Average dimensionless mixing free energy (Square init): {avg_fe_square:.6f}")
        plot_psi(psi_square, title=f"ψ at c=1 (Square init, r={P.r}, γ={P.gamma})")
        plot_reciprocal_lattice(psi_square, title=f"|FFT(ψ)|^2 at c=1 (Square init, r={P.r}, γ={P.gamma})")

    if return_energies:
        return avg_fe_hex, avg_fe_square
    else:
        print("\nComparison Summary at c=1.0:")
        print(f"Average dimensionless mixing free energy (Hexagonal init): {avg_fe_hex:.6f}")
        print(f"Average dimensionless mixing free energy (Square init): {avg_fe_square:.6f}")
        if avg_fe_square < avg_fe_hex:
            print("-> Square initial condition led to a state with lower average mixing free energy at c=1.")
        elif avg_fe_hex < avg_fe_square:
            print("-> Hexagonal initial condition led to a state with lower average mixing free energy at c=1.")
        else:
            print("-> Average mixing free energies are approximately equal.")

# --- Parameter Screening for c=1 ---
def screen_pfc_parameters_at_c1(r_val, gamma_val, omega_values, n_steps=500):
    """
    Screens combinations of P.omega values (keeping P.r and P.gamma fixed)
    to evaluate phase stability at c=1.0.

    Runs simulations at fixed c=1.0 with hexagonal and square psi initializations
    and compares the resulting average mixing free energies for each parameter set.

    Parameters
    ----------
    r_val : float
        Fixed P.r value to use.
    gamma_val : float
        Fixed P.gamma value to use.
    omega_values : list or array
        List of P.omega values to test.
    n_steps : int
        Number of simulation steps for each run in the comparison function.

    Returns
    -------
    pandas.DataFrame
        DataFrame containing the tested parameters and the resulting
        average mixing free energies for hexagonal and square initializations.
    """
    results = []
    original_omega = P.omega
    original_r = P.r
    original_gamma = P.gamma

    print(f"Starting parameter screening for Omega at c=1.0 (P.r={r_val}, P.gamma={gamma_val})...")

    # Temporarily set fixed P.r and P.gamma for this screening
    P.r = r_val
    P.gamma = gamma_val

    for omega_val in omega_values:
        # Temporarily set P.omega
        P.omega = omega_val

        try:
            # Run comparison and get energies
            avg_fe_hex, avg_fe_square = compare_phase_stability_at_c1(
                    n_steps=n_steps,
                    return_energies=True
                )
            results.append({
                "P.r": r_val,
                "P.gamma": gamma_val,
                "P.omega": omega_val,
                "Avg_FE_Hex": avg_fe_hex,
                "Avg_FE_Square": avg_fe_square,
                "FE_Diff_Hex_Minus_Square": avg_fe_hex - avg_fe_square # Positive means Square is lower energy (more stable mixing-wise)
            })
            print(f"  Tested omega={omega_val:.1f}. Avg FE Hex: {avg_fe_hex:.6f}, Avg FE Square: {avg_fe_square:.6f}")

        except Exception as e:
            print(f"  Simulation failed for omega={omega_val:.1f} with error: {e}")
            results.append({
                "P.r": r_val,
                "P.gamma": gamma_val,
                "P.omega": omega_val,
                "Avg_FE_Hex": np.nan,
                "Avg_FE_Square": np.nan,
                "FE_Diff_Hex_Minus_Square": np.nan
            })
        finally:
            # Restore original P.omega after each test
            P.omega = original_omega

    print("Parameter screening finished.")

    # Restore original P.r and P.gamma
    P.r = original_r
    P.gamma = original_gamma
    P.omega = original_omega # Ensure omega is also restored


    return pd.DataFrame(results)

# --- Example Screening Run for Omega at c=1 ---
# Define the ranges of parameters to test
r_fixed_c1 = P.r # Use current P.r
gamma_fixed_c1 = P.gamma # Use current P.gamma
omega_test_values_c1 = np.linspace(1.5e4, 2.0e4, 5) # Example: 5 values around current omega

# Run the screening
screening_results_omega_c1 = screen_pfc_parameters_at_c1(r_fixed_c1, gamma_fixed_c1, omega_test_values_c1, n_steps=50)

# Display the results, sorted by energy difference
# For c=1, we want Square to be more stable, meaning Avg_FE_Square < Avg_FE_Hex,
# which corresponds to FE_Diff_Hex_Minus_Square > 0.
display(screening_results_omega_c1.sort_values(by="FE_Diff_Hex_Minus_Square", ascending=False))

# Interpretation: A more POSITIVE value of "FE_Diff_Hex_Minus_Square" means
# the Average Mixing Free Energy for the Square initial condition was LOWER,
# suggesting the parameters favor the square lattice more (in terms of mixing energy) at c=1.

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(screening_results_omega_c0["P.omega"], screening_results_omega_c0["FE_Diff_Hex_Minus_Square"], marker='o', label='c = 0.0')
plt.plot(screening_results_omega_c1["P.omega"], screening_results_omega_c1["FE_Diff_Hex_Minus_Square"], marker='s', label='c = 1.0')

plt.xlabel("Omega (J/mol)")
plt.ylabel("FE_Diff_Hex_Minus_Square (Dimensionless)")
plt.title("Energy Difference (Hex - Square) vs Omega at c=0 and c=1")
plt.axhline(0, color='gray', linestyle='--', lw=0.8) # Line at zero difference
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# --- Coupled CH-PFC Simulation from Distributed System ---
ny, nx = 145, 145 # You can adjust the size as needed

# Initial concentration: uniform in spinodal region + random noise
initial_c_avg = 0.5 # Average concentration - choose in spinodal region for phase separation
initial_c_noise_amplitude = 0.1 # Amplitude of initial concentration noise
c0_distributed = np.full((ny, nx), initial_c_avg) + initial_c_noise_amplitude * (rng.random((ny, nx)) - 0.5)
c0_distributed = np.clip(c0_distributed, 0.0, 1.0) # Ensure concentration stays within [0, 1]

# Initial psi: random initialization
# Use the 'random' psi_init_method we added
psi_init_params_random = {"value_range": (-0.2, 0.2)} # Adjust range as needed

# Set dopant field (uniform dopant for simplicity, or adjust as needed)
C_dop_distributed = make_dopant_field(c0_distributed.shape, kind="uniform", value=P.C_dopant_scalar)

# Simulation parameters (use your desired parameters from Params)
n_steps_distributed = 500 # Number of simulation steps - adjust to see evolution
snapshot_every_distributed = 50 # Save snapshot every X steps

print("Starting coupled CH-PFC simulation from distributed system...")

# Run the coupled simulation
c_final_distributed, psi_final_distributed, snapshots_distributed = run_quasistatic_CHPFC(
    c0_distributed.copy(),
    n_steps=n_steps_distributed,
    snapshot_every=snapshot_every_distributed,
    C_dop_field=C_dop_distributed,
    psi_init_method="random", # Use the random initialization for psi
    psi_init_params=psi_init_params_random
)

print("Coupled CH-PFC simulation finished.")

# --- Plotting Snapshots ---
print("\nPlotting snapshots...")
for step, c_snapshot, psi_snapshot in snapshots_distributed:
    plot_concentration_2d(c_snapshot, title=f"c after CH-PFC (step {step})")
    plot_psi(psi_snapshot, title=f"ψ after CH-PFC (step {step})")
    # You might also want to use the combined plot function
    # plot_concentration_and_psi_contours(c_snapshot, psi_snapshot, title=f"c and ψ contours (step {step})")

# Plot final state details
plot_concentration_2d(c_final_distributed, title="Final c after CH-PFC")
plot_psi(psi_final_distributed, title="Final ψ after CH-PFC")
plot_reciprocal_lattice(psi_final_distributed, title="Final |FFT(ψ)|^2 (reciprocal space)")

# Optional: Plot final stress
# compute_and_plot_stress(c_final_distributed, "Final state")

In [ ]:
from scipy.ndimage import maximum_filter
from scipy.signal import find_peaks

def quantify_squareness(psi, dx=P.dx, prominence=0.1, distance=5):
    """
    Quantifies the degree of squareness of the lattice in the psi field
    by analyzing peaks in the reciprocal lattice.

    Parameters
    ----------
    psi : 2D array
        The psi field.
    dx : float
        Spatial step size.
    prominence : float
        Required prominence of peaks for scipy.signal.find_peaks.
    distance : int
        Required horizontal distance between peaks for scipy.signal.find_peaks.

    Returns
    -------
    dict
        A dictionary containing quantitative measures of squareness.
    """
    ny, nx = psi.shape
    Pk = np.fft.fftshift(np.abs(np.fft.fft2(psi))**2)

    # Find peaks in the power spectrum
    # Flatten the array to use find_peaks
    Pk_flat = Pk.ravel()
    # Find peaks in the flattened array
    peaks_indices_flat, properties = find_peaks(Pk_flat, prominence=prominence, distance=distance)

    # Convert flat indices back to 2D coordinates
    peaks_coords_flat = np.unravel_index(peaks_indices_flat, Pk.shape)
    peaks_coords = np.vstack(peaks_coords_flat).T # (n_peaks, 2) array of (y, x) coordinates

    # Exclude the central peak (DC component)
    center_y, center_x = ny // 2, nx // 2
    # Find index of the peak closest to the center
    distances_to_center = np.sqrt((peaks_coords[:, 0] - center_y)**2 + (peaks_coords[:, 1] - center_x)**2)
    if len(distances_to_center) > 0:
        central_peak_index = np.argmin(distances_to_center)
        # Remove the central peak
        peaks_coords = np.delete(peaks_coords, central_peak_index, axis=0)
        peaks_indices_flat = np.delete(peaks_indices_flat, central_peak_index)
        peak_magnitudes = Pk_flat[peaks_indices_flat]
    else:
        # No peaks found other than possibly the center
        return {
            "num_peaks_found": 0,
            "square_peak_count": 0,
            "hexagonal_peak_count": 0,
            "squareness_ratio": 0.0,
            "peak_distance_variation": np.nan,
            "peak_angle_variation": np.nan
        }


    # Sort peaks by magnitude (brightness) in descending order
    sorted_indices = np.argsort(peak_magnitudes)[::-1]
    sorted_peaks_coords = peaks_coords[sorted_indices]
    sorted_peak_magnitudes = peak_magnitudes[sorted_indices]

    # Analyze the top few brightest peaks (e.g., top 4 or 6)
    num_peaks_to_analyze = min(6, len(sorted_peaks_coords)) # Look at up to 6 brightest peaks
    analyzed_peaks_coords = sorted_peaks_coords[:num_peaks_to_analyze]
    analyzed_peak_magnitudes = sorted_peak_magnitudes[:num_peaks_to_analyze]

    # Convert pixel coordinates to wavevectors (approximate)
    # Need to map pixel coordinates (y, x) relative to center to (ky, kx)
    ky_vals = np.fft.fftfreq(ny, d=dx) * 2*np.pi
    kx_vals = np.fft.fftfreq(nx, d=dx) * 2*np.pi # Note: rfftfreq for rfftn

    # Get the k-space coordinates of the analyzed peaks
    analyzed_peaks_k = []
    for p_coord in analyzed_peaks_coords:
        py, px = p_coord
        # Map pixel index relative to center to k-value
        # The kx_vals and ky_vals arrays are already sorted for fftfreq
        # Need to handle the shift correctly
        k_y = ky_vals[py if py < ny // 2 else py - ny]
        k_x = kx_vals[px if px < nx // 2 else px - nx] # This is simplified, need to be careful with fftshift and rfft
        # A more direct way might be to get the k-values corresponding to the pixel indices after fftshift
        k_y_shifted = np.fft.fftshift(ky_vals)[py]
        k_x_shifted = np.fft.fftshift(kx_vals)[px]

        analyzed_peaks_k.append((k_x_shifted, k_y_shifted)) # Store as (kx, ky)


    analyzed_peaks_k = np.array(analyzed_peaks_k)

    # --- Quantitative Measures ---

    # 1. Number of dominant peaks (excluding center)
    num_peaks_found = len(peaks_coords)

    # 2. Check for square pattern (4 peaks at approx (k0,0), (-k0,0), (0,k0), (0,-k0))
    #    and hexagonal pattern (6 peaks with 60 deg separation)
    square_peak_count = 0
    hexagonal_peak_count = 0
    k_magnitudes = np.sqrt(analyzed_peaks_k[:, 0]**2 + analyzed_peaks_k[:, 1]**2)
    k0_approx = np.mean(k_magnitudes) if len(k_magnitudes) > 0 else np.nan

    if len(analyzed_peaks_k) >= 4 and not np.isnan(k0_approx):
         # Check for approximate square arrangement
         # Look for peaks close to axes
         angles = np.arctan2(analyzed_peaks_k[:, 1], analyzed_peaks_k[:, 0]) # angles in radians
         angles_deg = np.degrees(angles) % 360

         # Check if angles are close to 0, 90, 180, 270 deg
         square_angles = np.array([0, 90, 180, 270])
         angle_tolerance = 15 # degrees

         is_square_like = False
         if len(analyzed_peaks_k) >= 4:
             # Check if at least 4 peaks have angles close to square angles
             angles_diff = np.abs(angles_deg[:, None] - square_angles[None, :])
             min_angle_diff = np.min(angles_diff, axis=1)
             peaks_close_to_square_angles = np.sum(min_angle_diff < angle_tolerance)
             if peaks_close_to_square_angles >= 4:
                 is_square_like = True
                 square_peak_count = peaks_close_to_square_angles # More accurate count

         # Check for approximate hexagonal arrangement (6 peaks with 60 deg separation)
         hexagonal_angles = np.array([0, 60, 120, 180, 240, 300])
         if len(analyzed_peaks_k) >= 6:
             angles_diff_hex = np.abs(angles_deg[:, None] - hexagonal_angles[None, :])
             min_angle_diff_hex = np.min(angles_diff_hex, axis=1)
             peaks_close_to_hex_angles = np.sum(min_angle_diff_hex < angle_tolerance)
             if peaks_close_to_hex_angles >= 6:
                 hexagonal_peak_count = peaks_close_to_hex_angles


    # 3. Ratio of square peak intensity to total dominant peak intensity
    #    This is a simple measure, could be refined.
    squareness_ratio = 0.0
    if square_peak_count > 0:
         # Sum magnitudes of peaks identified as square-like
         square_like_peak_magnitudes = analyzed_peak_magnitudes[min_angle_diff < angle_tolerance]
         total_analyzed_magnitude = np.sum(analyzed_peak_magnitudes)
         if total_analyzed_magnitude > 1e-12:
             squareness_ratio = np.sum(square_like_peak_magnitudes) / total_analyzed_magnitude
         else:
              squareness_ratio = 0.0


    # 4. Variation in distances of the main peaks from the origin (should be ~k0)
    peak_distance_variation = np.std(k_magnitudes) / k0_approx if k0_approx > 1e-12 and len(k_magnitudes) > 1 else np.nan

    # 5. Variation in angles between main peaks (should be ~90 deg for square, ~60 for hex)
    #    This is harder to quantify robustly without pairing peaks.
    #    A simpler approach is to look at the deviation of angles from ideal square angles.
    if is_square_like and len(analyzed_peaks_k) >= 4:
         # Calculate angles between consecutive peaks if sorted by angle, or more robustly check angles between pairs
         # For simplicity, let's just report the min angle diff to ideal square angles for square-like peaks
         peak_angle_variation = np.mean(min_angle_diff[min_angle_diff < angle_tolerance]) if np.sum(min_angle_diff < angle_tolerance) > 0 else np.nan
    else:
         peak_angle_variation = np.nan


    results = {
        "num_peaks_found": num_peaks_found, # Total non-central peaks found
        "square_peak_count": square_peak_count, # Peaks close to square angles
        "hexagonal_peak_count": hexagonal_peak_count, # Peaks close to hexagonal angles
        "squareness_ratio": squareness_ratio, # Ratio of magnitude of square-like peaks to total analyzed peaks
        "peak_distance_variation": peak_distance_variation, # Std deviation of k-magnitudes / mean k-magnitude
        "peak_angle_variation": peak_angle_variation # Mean deviation from ideal square angles for square-like peaks
    }

    return results

# --- Example Usage ---
# Assuming psi_u is available from a simulation run
# squareness_metrics = quantify_squareness(psi_u)
# print("\nSquareness Metrics:")
# print(squareness_metrics)

# Example with gradient psi (if available)
# squareness_metrics_g = quantify_squareness(psi_g)
# print("\nSquareness Metrics (Gradient):")
# print(squareness_metrics_g)

In [ ]:
# Assuming psi_u is available from a simulation run
squareness_metrics = quantify_squareness(psi_u)
print("\nSquareness Metrics:")
print(squareness_metrics)

In [ ]:
# --- Coupled CH-PFC Simulation from Distributed System ---
ny, nx = 145, 145 # You can adjust the size as needed

# Initial concentration: uniform in spinodal region + random noise
initial_c_avg = 0.5 # Average concentration - choose in spinodal region for phase separation
initial_c_noise_amplitude = 0.1 # Amplitude of initial concentration noise
c0_distributed = np.full((ny, nx), initial_c_avg) + initial_c_noise_amplitude * (rng.random((ny, nx)) - 0.5)
c0_distributed = np.clip(c0_distributed, 0.0, 1.0) # Ensure concentration stays within [0, 1]

# Initial psi: random initialization
# Use the 'random' psi_init_method we added
psi_init_params_random = {"value_range": (-0.2, 0.2)} # Adjust range as needed

# Set dopant field (uniform dopant for simplicity, or adjust as needed)
C_dop_distributed = make_dopant_field(c0_distributed.shape, kind="uniform", value=P.C_dopant_scalar)

# Simulation parameters (use your desired parameters from Params)
n_steps_distributed = 500 # Number of simulation steps - adjust to see evolution
snapshot_every_distributed = 50 # Save snapshot every X steps

print("Starting coupled CH-PFC simulation from distributed system...")

# Run the coupled simulation
c_final_distributed, psi_final_distributed, snapshots_distributed = run_quasistatic_CHPFC(
    c0_distributed.copy(),
    n_steps=n_steps_distributed,
    snapshot_every=snapshot_every_distributed,
    C_dop_field=C_dop_distributed,
    psi_init_method="random", # Use the random initialization for psi
    psi_init_params=psi_init_params_random
)

print("Coupled CH-PFC simulation finished.")

# --- Plotting Snapshots ---
print("\nPlotting snapshots...")
for step, c_snapshot, psi_snapshot in snapshots_distributed:
    plot_concentration_2d(c_snapshot, title=f"c after CH-PFC (step {step})")
    plot_psi(psi_snapshot, title=f"ψ after CH-PFC (step {step})")
    # You might also want to use the combined plot function
    # plot_concentration_and_psi_contours(c_snapshot, psi_snapshot, title=f"c and ψ contours (step {step})")

# Plot final state details
plot_concentration_2d(c_final_distributed, title="Final c after CH-PFC")
plot_psi(psi_final_distributed, title="Final ψ after CH-PFC")
plot_reciprocal_lattice(psi_final_distributed, title="Final |FFT(ψ)|^2 (reciprocal space)")

# Optional: Plot final stress
# compute_and_plot_stress(c_final_distributed, "Final state")